In [1]:
import json
import pickle
import time
from tqdm import tqdm
import requests
import re
import pandas as pd

In [2]:
kgdf = pd.read_pickle("kgdf.pickle")

In [3]:
kgdf["entity1"] = kgdf.entity1.apply(lambda x: x.strip())
kgdf["relation"] = kgdf.relation.apply(lambda x: x.strip())
kgdf["entity2"] = kgdf.entity2.apply(lambda x: x.strip())

In [4]:
kgdf_clean = kgdf[~((kgdf["entity1"] == "") | (kgdf["relation"] == "") | (kgdf["entity2"] == ""))]

In [5]:
triplets = kgdf_clean.values

In [6]:
triplets[0]

array(['http://dbpedia.org/resource/Headhunter_(2009_film)', 'country',
       'http://dbpedia.org/resource/Denmark'], dtype=object)

In [40]:
!pip install pykeen==1.0.5

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached optuna-2.10.0-py3-none-any.whl (308 kB)
  Using cached click_default_group-1.2.2-py3-none-any.whl
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached alembic-1.7.7-py3-none-any.whl (210 kB)
  Using cached colorlog-6.6.0-py2.py3-none-any.whl (11 kB)
  Using cached PyYAML-6.0-cp37-cp37m-win_amd64.whl (153 kB)
  Using cached cliff-3.10.1-py3-none-any.whl (81 kB)
  Using cached SQLAlchemy-1.4.35-cp37-cp37m-win_amd64.whl (1.6 MB)
  Using cached cmaes-0.8.2-py3-none-any.whl (15 kB)
  Using cached greenlet-1.1.2-cp37-cp37m-win_amd64.whl (101 kB)
  Using cached Mako-1.2.0-py3-none-any.whl (78 kB)
  Using cached prettytable-3.2.0-py3-none-any.whl (26 kB)
  Using cached autopage-0.5.0-py3-none-any.whl (29 kB)
  Using cached stevedore-3.5.0-py3-none-any.whl (49 kB)
  Using cached pbr-5.8.1-py2.py3-none-any.whl (113 kB)
  Using cached pyperclip-1.8.2-py3-n

In [7]:
from pykeen.datasets import OpenBioLink
dataset = OpenBioLink()
dataset.training.triples

C:\Users\Ashwin\Anaconda3\envs\chat\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([['NCBIGENE:11200', 'GENE_PHENOTYPE', 'HP:0009919'],
       ['NCBIGENE:2649', 'GENE_EXPRESSED_ANATOMY', 'UBERON:0000059'],
       ['NCBIGENE:534', 'GENE_EXPRESSED_ANATOMY', 'UBERON:0000467'],
       ...,
       ['NCBIGENE:1269', 'GENE_REACTION_GENE', 'NCBIGENE:6376'],
       ['NCBIGENE:1785', 'GENE_EXPRESSED_ANATOMY', 'UBERON:0003729'],
       ['NCBIGENE:55591', 'GENE_EXPRESSED_ANATOMY', 'UBERON:0002038']],
      dtype=object)

In [8]:
def split_kg(df, test_split=0.3, error=50):
    epairs = {}
    for idx, (e1, e2) in df[["entity1", "entity2"]].iterrows():
        if (e1, e2) in epairs:
            key = (e1, e2)
        elif (e2, e1) in epairs:
            key = (e2, e1)
        else:
            key = (e1, e2)
            epairs[key] = []
        epairs[key].append(idx)
    epairs_sorted = {k: (v, len(v)) for k, v in sorted(x.items(), key=lambda item: len(item[1]), reverse=True)}
    total_size = df.shape[0]
    test_size = test_split*total_size
    test_ids = []
    for epair, value in epairs_sorted:
        ids, ids_len = value
        if len(test_ids) > test_size:
            break
        remaining_size = test_size - len(test_ids)
        if ids_len < remaining_size + error:
            test_ids += ids
    print(len(test_ids))
    print(len(set(test_ids)))
        
    

In [9]:
train, test = split_kg(kgdf)

NameError: name 'x' is not defined